In [1]:
import plotly.graph_objs as go  # 載入plotly套件
import numpy as np              # 載入numpy套件
import pandas as pd             # 載入pandas套件
import datetime as dt           # 載入datetime套件
from datetime import datetime
from plotly.subplots import make_subplots  #載入plotly畫子圖的套件

In [2]:
profit_df = pd.read_excel('每日損益紀錄.xlsx')
profit_df.head(5)

,日期,每日損益資金(元)
0,1998-09-16,8199.836
1,1998-10-21,2799.944
2,1998-11-18,14999.700
3,1998-12-11,-1199.976
4,1998-12-16,-12999.740


In [3]:
#初始化資金1000000
capital = 1000000
# 初始化策略累積資金
profit_df.loc[0, '策略累積資金(金額)'] = capital

# 開始累積
for i in range(1, len(profit_df)):
    profit_df.loc[i, '策略累積資金(金額)'] = profit_df.loc[i-1, '策略累積資金(金額)']+profit_df.loc[i, '每日損益資金(元)']

profit_df.head(5)

,日期,每日損益資金(元),策略累積資金(金額)
0,1998-09-16,8199.836,1000000.000
1,1998-10-21,2799.944,1002799.944
2,1998-11-18,14999.700,1017799.644
3,1998-12-11,-1199.976,1016599.668
4,1998-12-16,-12999.740,1003599.928


In [4]:
# 計算每日報酬率
profit_df['策略每日報酬率'] = profit_df['策略累積資金(金額)'].pct_change()
profit_df.loc[0, '策略每日報酬率'] = 0  # 初始化第一天策略每日報酬率

# 計算累積報酬率
profit_df['策略累積報酬率'] = (1 + profit_df['策略每日報酬率']).cumprod()-1
profit_df.loc[0, '策略累積報酬率'] = 0  # 初始化第一天策略累積報酬率

# 計算Drawdown
# 計算每個時間點的最大價格
profit_df['每個時點最大策略累積資金(金額)'] = profit_df['策略累積資金(金額)'].cummax()
# 計算每個時間點的 Drawdown
profit_df['Drawdown'] = (profit_df['策略累積資金(金額)'] - profit_df['每個時點最大策略累積資金(金額)']) / capital

In [5]:
profit_df

,日期,每日損益資金(元),策略累積資金(金額),策略每日報酬率,策略累積報酬率,每個時點最大策略累積資金(金額),Drawdown
0,1998-09-16,8199.836,1000000.000,0.000000,0.000000,1000000.000,0.000000
1,1998-10-21,2799.944,1002799.944,0.002800,0.002800,1002799.944,0.000000
2,1998-11-18,14999.700,1017799.644,0.014958,0.017800,1017799.644,0.000000
3,1998-12-11,-1199.976,1016599.668,-0.001179,0.016600,1017799.644,-0.001200
4,1998-12-16,-12999.740,1003599.928,-0.012787,0.003600,1017799.644,-0.014200
...,...,...,...,...,...,...,...
418,2024-02-20,-16399.672,1857182.856,-0.008753,0.857183,1956780.864,-0.099598
419,2024-03-19,-23799.524,1833383.332,-0.012815,0.833383,1956780.864,-0.123398
420,2024-04-16,56998.860,1890382.192,0.031089,0.890382,1956780.864,-0.066399
421,2024-05-14,25999.480,1916381.672,0.013754,0.916382,1956780.864,-0.040399


#### 計算Max Drawdown

In [6]:
# 定義繪製策略損益圖方法
def draw_PnL(trace1, trace2,  title1, title2):
    
    # 打開子畫布
    fig = make_subplots(rows=2, cols=1, subplot_titles=(title1, title2), row_heights=[0.75, 0.25], shared_xaxes=True, vertical_spacing=0.1)
    
    # 帶入資料到子畫布的特定位置
    fig.add_trace(trace1, row=1, col=1)
    fig.add_trace(trace2, row=2, col=1)
    
    # 設定子畫布共同屬性
    fig.update_layout(
        title='台指期近月指數肯特納通道策略', # 設定圖表標題名稱
        plot_bgcolor='white', # 設定底色為白色
    )
    
    # 設定子畫布屬性
    fig.update_xaxes(row=1, col=1, linecolor='black', linewidth=2, showgrid=True, gridcolor='lightgrey')
    fig.update_yaxes(title_text="損益(%)", row=1, col=1, linecolor='black', linewidth=2, showgrid=True, gridcolor='lightgrey', zerolinecolor='darkgrey')
    fig.update_xaxes(title_text="日期", row=2, col=1, linecolor='black', linewidth=2, showgrid=True, gridcolor='lightgrey')
    fig.update_yaxes(title_text="損益(%)", row=2, col=1, linecolor='black', linewidth=2, showgrid=True, gridcolor='lightgrey', zerolinecolor='darkgrey')

    #展示圖表
    fig.show()
    
    return

In [7]:
# 將日期轉換為 datetime 物件
profit_df['日期'] = pd.to_datetime(profit_df['日期'])

**<H>特殊格式<H>**

In [8]:
# 建立plotly損益曲線
trace1 = go.Scatter(x=profit_df['日期'].dt.strftime('%Y-%m'),  # 格式化為年月
                    y=profit_df['策略累積報酬率']*100, 
                    mode='lines', 
                    name='策略損益曲線', 
                    line=dict(color='blue')
                    )
trace2 = go.Scatter(x=profit_df['日期'].dt.strftime('%Y-%m'),  # 格式化為年月
                    y=profit_df['Drawdown']*100, 
                    mode='lines', 
                    name='Drawdown', 
                    line=dict(color='red')
                    )
# 繪製策略損益圖
draw_PnL(trace1, trace2, '台指期月結算期貨損益', 'Drawdown')

**<H>一般格式<H>**

In [9]:
# 建立plotly損益曲線
trace1 = go.Scatter(x=profit_df['日期'], 
                    y=profit_df['策略累積報酬率']*100, 
                    mode='lines', 
                    name='策略損益曲線', 
                    line=dict(color='blue')
                    )
trace2 = go.Scatter(x=profit_df['日期'], 
                    y=profit_df['Drawdown']*100, 
                    mode='lines', 
                    name='Drawdown', 
                    line=dict(color='red')
                    )
# 繪製策略損益圖
draw_PnL(trace1, trace2, '台指期月結算期貨損益', 'Drawdown')

In [10]:
# 計算績效指標
total_return = profit_df.loc[len(profit_df)-1, '策略累積報酬率']
annualized_return = (1+profit_df.loc[len(profit_df)-1, '策略累積報酬率'])**(252/len(profit_df))-1  # 年化報酬率
annualized_volatility = np.std(profit_df['策略每日報酬率'])*np.sqrt(252)  # 年化波動率
sharpe_ratio = annualized_return/annualized_volatility  # 年化夏普值
max_drawdown = profit_df['Drawdown'].min()  # 最大回撤(MDD)

print("總報酬率: {}%".format(round(total_return*100,2)))
print("年化報酬率: {}%".format(round(annualized_return*100,2)))
print("年化波動率: {}%".format(round(annualized_volatility*100,2)))
print("年化夏普值: {}".format(round(sharpe_ratio,2))) 
print("最大回撤(MDD): {}%".format(round(max_drawdown*100,2)))

總報酬率: 100.44%
年化報酬率: 51.32%
年化波動率: 25.23%
年化夏普值: 2.03
最大回撤(MDD): -43.36%
